In [ ]:
#IMPORTS

import numpy as np
import random
import tensorflow as tf
import tensorflow.keras as kr
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist

from scipy.spatial.distance import euclidean
from sklearn.metrics import confusion_matrix

from time import sleep
from tqdm import tqdm

import copy

import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
from numpy.random import RandomState
import scipy as scp
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras import optimizers
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.utils import to_categorical
from keras import backend as K
from itertools import product
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn import mixture

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
rs = RandomState(92) #To reproduce the same results each time we run this notebook

In [ ]:
#Load dataset into a pandas DataFrame
adult_data = pd.read_csv('adult_data.csv', na_values='?')

## Preliminary Data Analysis

In [ ]:
# Drop all records with missing values
adult_data.dropna(inplace=True)
adult_data.reset_index(drop=True, inplace=True)

#Data columns and their types
adult_data.info()
adult_data.head(10)

### Unique values of each attribute:

In [ ]:
adult_data.nunique()

### Bar plot of each attribute:

In [ ]:
fig = plt.figure(figsize=(20,15))
cols = 5
rows = math.ceil(float(adult_data.shape[1]) / cols)
for i, column in enumerate(adult_data.columns):
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(column)
    if adult_data.dtypes[column] == np.object:
        adult_data[column].value_counts().plot(kind="bar", axes=ax)
    else:
        adult_data[column].hist(axes=ax)
        plt.xticks(rotation="vertical")
plt.subplots_adjust(hspace=0.9, wspace=0.2)

### Income attribute:

In [ ]:
print(adult_data.income.value_counts())
sns.countplot(x='income', data=adult_data)
plt.show()

In [ ]:
percent = ((adult_data.income.value_counts()) * 100)/len(adult_data)
print(percent)

### Race attribute:

In [ ]:
adult_data['race'].value_counts()

In [ ]:
percent1 = ((adult_data.race.value_counts()) * 100)/len(adult_data)
percent1

In [ ]:
percent2 = ((adult_data.groupby('race')['income'].value_counts()) * 100)/len(adult_data)
percent2

### Plotting 'race' according to 'income':

In [ ]:
race = round(pd.crosstab(adult_data.race, adult_data.income).div(pd.crosstab(adult_data.race, adult_data.income).apply(sum,1),0),2)
race.sort_values(by = '>50K', inplace = True)
ax = race.plot(kind ='bar', color=['#eca0d7','#64c774'], title = 'Proportion distribution across race levels', figsize = (10,8))
ax.set_xlabel('Race level')
ax.set_ylabel('Proportion of population')

print()

#### The 'educational-num' column can be dropped since it's the same as'education'. And the columns 'capital-gain' and 'capital-loss' have more zeroes: 

In [ ]:
adult_data.drop(['educational-num', 'capital-gain', 'capital-loss'], axis=1, inplace = True)

#### Next, we replace 'income' values with 0 and 1, and the same for 'race. Thus, we have 0 for 'Black' and 1 for 'nonblacks'.


In [ ]:
adult_data.replace(['<=50K','>50K'],
             [0,1], inplace = True)

In [ ]:
adult_data.replace(['Black', 'White','Other','Amer-Indian-Eskimo','Asian-Pac-Islander'],
             [0,1,1,1,1], inplace = True)

## Data Preparation

One-hot encoding is the process of representing multi-class categorical features as binary features, one for each class. Although this process increases the dimensionality of the dataset, classification algorithms tend to work better on this format of data.

I use one-hot encoding to represent all the categorical features in the dataset. 


In [ ]:
category_col_1 =[ 'education', 'occupation',
               'relationship','native-country','workclass','marital-status', 'gender'] 

adult = pd.get_dummies(adult_data, columns=category_col_1, drop_first=True)

In [ ]:
dataframe=adult.drop('fnlwgt',1)
dataframe =dataframe[[c for c in dataframe if c not in ['income']] + ['income']]

In [ ]:
X = dataframe.iloc[:, 0:95].values
y = dataframe.iloc[:, 95].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))
print(X_train.shape)

#### Normalization:


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### Training with neural nets:

In [ ]:
# BASELINE SCENARIO
def build_model():
    model = Sequential()

    # Adding the input layer and the first hidden layer
    model.add(Dense(output_dim = 70, activation = 'relu', input_dim = 93))
    # Adding the second hidden layer
    model.add(Dense(output_dim = 50, activation = 'relu'))
    # Adding the output layer
    model.add(Dense(output_dim = 1, activation = 'sigmoid'))

    opt = keras.optimizers.Adam(learning_rate=3e-4)
    model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

    model.fit(X_train, y_train, batch_size = 10, nb_epoch = 100, validation_split=0.2)
    return model

model = build_model()

#### Confusion matrix for 'income':

In [ ]:
pred = model.predict(X_test)
y_pred = np.where(pred>=0.5, 1,0)

from sklearn.metrics import confusion_matrix
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
tn,fp,fn,tp = cm.ravel()

#### Performance measures:

In [ ]:
#accuracy: (tp + tn)/(tp + tn + fp + fn)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
#precision: tp/tp+fp
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
#recall: tp/tp+fn
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
#FNR: 1-recall
fnr = 1 - recall
print('FNR: %f' % fnr)
#FPR: fp/fp+tn
fpr = fp / (fp + tn)
print('FPR: %f' % fpr)
#f1: 2 tp/ (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)


#### ROC AUC score and Gini coefficient:

In [ ]:
from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)


plt.title('Receiver Operating Characteristic Curve')
plt.plot(false_positive_rate, true_positive_rate, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Gini_coefficient=2*roc_auc - 1
print ("Gini_coefficient =",Gini_coefficient)

#### Confusion matrix for 'Black' (0) individuals:

In [ ]:
filter_rows = X_test[:,2]==0
X_test_black = X_test[filter_rows,:]
y_test_black = y_test[filter_rows]
y_pred_black = model.predict(X_test_black)
y_pred_b = np.where(y_pred_black>=0.5, 1,0)

print(classification_report(y_test_black, y_pred_b))
cmb = confusion_matrix(y_test_black, y_pred_b)

#### Performance measures:

In [ ]:
#accuracy: (tp + tn)/(tp + tn + fp + fn)
accuracy = accuracy_score(y_test_black, y_pred_b)
print('Accuracy: %f' % accuracy)
#precision: tp/tp+fp
precision = precision_score(y_test_black, y_pred_b)
print('Precision: %f' % precision)
#recall: tp/tp+fn
recall = recall_score(y_test_black, y_pred_b)
print('Recall: %f' % recall)
#FNR: 1-recall
fnr = 1 - recall
print('FNR: %f' % fnr)
#FPR: fp/fp+tn
fpr = fp / (fp + tn)
print('FPR: %f' % fpr)
#f1: 2 tp/ (2 tp + fp + fn)
f1 = f1_score(y_test_black, y_pred_b)
print('F1 score: %f' % f1)

#### ROC AUC score and Gini coefficient:

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_black, y_pred_b)
roc_auc = auc(false_positive_rate, true_positive_rate)


plt.title('Receiver Operating Characteristic Curve')
plt.plot(false_positive_rate, true_positive_rate, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Gini_coefficient=2*roc_auc - 1
print ("Gini_coefficient =",Gini_coefficient)

#### Confusion matrix for 'nonblack' (1) individuals:

In [ ]:
filter_rows = X_test[:,2]==1
X_test_nonblack = X_test[filter_rows,:]
y_test_nonblack = y_test[filter_rows]
y_pred_nonblack = model.predict(X_test_nonblack)
y_pred_nb = np.where(y_pred_nonblack>=0.5, 1,0)

print(classification_report(y_test_nonblack, y_pred_nb))
cmnb = confusion_matrix(y_test_nonblack, y_pred_nb)

#### Performance measures: 

In [ ]:
#accuracy: (tp + tn)/(tp + tn + fp + fn)
accuracy = accuracy_score(y_test_nonblack, y_pred_nb)
print('Accuracy: %f' % accuracy)
#precision: tp/tp+fp
precision = precision_score(y_test_nonblack, y_pred_nb)
print('Precision: %f' % precision)
#recall: tp/tp+fn
recall = recall_score(y_test_nonblack, y_pred_nb)
print('Recall: %f' % recall)
#FNR: 1-recall
fnr = 1 - recall
print('FNR: %f' % fnr)
#FPR: fp/fp+tn
fpr = fp / (fp + tn)
print('FPR: %f' % fpr)
#f1: 2 tp/ (2 tp + fp + fn)
f1 = f1_score(y_test_nonblack, y_pred_nb)
print('F1 score: %f' % f1)

#### ROC AUC score and Gini coefficient:

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_nonblack, y_pred_nb)
roc_auc = auc(false_positive_rate, true_positive_rate)


plt.title('Receiver Operating Characteristic Curve')
plt.plot(false_positive_rate, true_positive_rate, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Gini_coefficient=2*roc_auc - 1
print ("Gini_coefficient =",Gini_coefficient)